# Find the vertex

In [1]:
import cv2
import numpy as np

In [2]:
img = cv2.imread('photo1.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
dilate = cv2.dilate(blurred, cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))) # create a kernel of size 3 x 3 and dilate image
edged = cv2.Canny(dilate, 30, 120, 3) # edge detection

cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  # retrieve only the extreme outer contours and compress horizontal, vertical, and diagonal segments and leaves only their end points
cnts = cnts[0] # image
docCnt = None

if len(cnts) > 0:
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    for c in cnts:
        peri = cv2.arcLength(c, True) # arc length
        approx = cv2.approxPolyDP(c, 0.02*peri, True) # approximate a curve/polygon with another curve/polygon with less vertices o that the distance between them is less or equal to the specified precision
        if len(approx) == 4:
            docCnt = approx
            break
print(docCnt)
for peak in docCnt:
    peak = peak[0]
    cv2.circle(img, tuple(peak), 10, (255, 0, 0))
    
#cv2.imshow('img', img)
#cv2.waitKey(0)

[[[207 151]]

 [[ 16 603]]

 [[344 732]]

 [[518 283]]]


# Perspective Transformation

In [3]:
img1 = cv2.imread('photo1.jpg')
result = img1.copy()

src = np.float32([[207, 151], [16, 603], [344, 732], [518, 283]])
dst = np.float32([[0, 0], [0, 500], [400, 500], [400, 0]])
print(img1.shape)
m = cv2.getPerspectiveTransform(src, dst)
print("warpMatrix:")
print(m)
result1 = cv2.warpPerspective(result, m, (400, 500))
cv2.imshow("src",img1)
cv2.imshow("result",result1)
cv2.waitKey()

(960, 540, 3)
warpMatrix:
[[ 1.06163179e+00  4.48609893e-01 -2.87497874e+02]
 [-4.12062052e-01  9.70843167e-01 -6.13004736e+01]
 [-8.42123816e-05  6.03604064e-05  1.00000000e+00]]


-1

# WarpMatrix

In [4]:
def WarpPerspectiveMatrix(src, dst):
    assert src.shape[0] == dst.shape[0] and src.shape[0] >= 4
    
    nums = src.shape[0]
    A = np.zeros((2*nums, 8))
    B = np.zeros((2*nums, 1))
    for i in range(0, nums):
        A_i = src[i,:]
        B_i = dst[i,:]
 
        A[2*i, :] = [A_i[0], A_i[1], 1, 0, 0, 0, -A_i[0]*B_i[0], -A_i[1]*B_i[0]]
        B[2*i] = B_i[0]
        
        A[2*i+1, :] = [0, 0, 0, A_i[0], A_i[1], 1, -A_i[0]*B_i[1], -A_i[1]*B_i[1]]
        B[2*i+1] = B_i[1]

    A = np.mat(A)
    warpMatrix = A.I * B
    warpMatrix = np.array(warpMatrix).T[0]
    warpMatrix = np.insert(warpMatrix, warpMatrix.shape[0], values=1.0, axis=0)
    warpMatrix = warpMatrix.reshape((3, 3))
    return warpMatrix

if __name__ == '__main__':
    print('warpMatrix')
    src = np.array([[10.0, 457.0], [395.0, 291.0], [624.0, 291.0], [1000.0, 457.0]])
    dst = np.array([[46.0, 920.0], [46.0, 100.0], [600.0, 100.0], [600.0, 920.0]])
    
    warpMatrix = WarpPerspectiveMatrix(src, dst)
    print(warpMatrix)

warpMatrix
[[-5.01338334e-01 -1.35357643e+00  5.82386716e+02]
 [-1.66533454e-15 -4.84035391e+00  1.38781980e+03]
 [-4.33680869e-19 -4.14856327e-03  1.00000000e+00]]
